# CS295/395: Secure Distributed Computation
## Homework 7

In [ ]:
# Imports and definitions
import numpy as np
import galois
import random
bits = 128

from collections import defaultdict
from collections import namedtuple

## Definitions: Paillier Cryptosystem

In [ ]:
def keygen(bits):
    """Generates keys with `bits`-bits of security. Returns a pair: (secret key, public key)."""
    def invmod(x, m):
        gcd, s, t = galois.egcd(x, m)
        assert gcd == 1
        return s

    p = galois.random_prime(int(bits/2))
    q = galois.random_prime(int(bits/2))

    n = p*q
    g = n+1
    lamb = (p-1) * (q-1)
    mu = invmod(lamb, n)
    
    sk = (lamb, mu)
    pk = (n, g)
    return sk, pk

def encrypt(m, pk):
    """Encrypts the message `m` with public key `pk`."""
    n, g = pk
    n_sq = n**2
    r = random.randint(1, n)
    c = (pow(g, m, n_sq) * pow(r, n, n_sq)) % n_sq
    return c

def decrypt(c, sk, pk):
    """Decrypts the ciphertext `c` using secret key `sk` and public key `pk`."""
    lamb, mu = sk
    n, g = pk
    n_sq = n**2
    L_result = (pow(c, lamb, n_sq) - 1)//n
    return (L_result * mu) % n

def e_add(c1, c2, pk):
    """Add one encrypted integer to another"""
    n, g = pk

    return c1 * c2 % n**2

## Question 1 (30 points)

Implement an electronic voting system using the Paillier cryptosystem. The system has two parts: a voter encrypts their vote using the cryptosystem and submits it to the server, and the server stores the encrypted votes and tallies them at the end of the election. The election server holds the public and private keys; the public key is available to the voters, so that they can encrypt their votes. Each voter should be able to verify that the server has correctly stored their vote, by consulting the encrypted votes to check that their vote indeed appears. Generate your public and private keys with at least 32-bit security.

*Reference:* [Prêt à Voter with Paillier encryption](https://dl.acm.org/doi/10.1016/j.mcm.2008.05.015).

In [ ]:
voting_table = ['Candidate 1',
                'Candidate 2',
                'Candidate 3']

class Voter:
    def vote(self, candidate, server):
        """Submits an encrypted vote to the server"""
        # YOUR CODE HERE
        raise NotImplementedError()

class ElectionServer:
    def __init__(self):
        self.votes = []
        self.sk, self.pk = keygen(32)
    
    def get_public_key(self):
        """Get the public key from the election server"""
        return self.pk
    
    def submit_vote(self, ct_vote_vector):
        """Submit an (encrypted) vote to the election server"""
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def show_votes(self):
        """Show the submitted (encrypted) votes"""
        return self.votes
    
    def tally_votes(self):
        """Tally up the votes at the end of the election. Returns a list of totals in the 
        same order as the list of candidates."""
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE
es = ElectionServer()

Voter().vote('Candidate 1', es)
Voter().vote('Candidate 2', es)
Voter().vote('Candidate 2', es)
Voter().vote('Candidate 3', es)

#print(e.show_billboard())
assert es.tally_votes() == [1, 2, 1]

## Question 2 (10 points)

In 2-5 sentences, answer the following:

- What trust assumptions do we make about the *election server* in this election system?
- What trust assumptions do we make about the *voter* in this election system?

YOUR ANSWER HERE

## Question 3 (10 points)

In 2-5 sentences, answer the following:

- What is one way a malicious *election server* could break the rules of the election?
- What are two ways a malicious *voter* could break the rules of the election?

YOUR ANSWER HERE

## Question 4 (30 points)

The [Boston Women's Workforce Council Gender Pay Gap Survey](https://thebwwc.org/mpc) uses MPC to deploy a survey of Boston-area businesses to determine how women and men are paid differently. Each business submits encrypted values for their employees' salaries, and the system calculates the average salaries for women and men across all of the businesses. This design protects the privacy of individual employees, and protects individual businesses from embarrassment.

Implement a system for conducting a survey like this using the Paillier cryptosystem. Participants should submit their own salaries, and specify their gender. The survey server should collect responses, and at the end of the survey, calculate the average salaries for men and women. Use at least 32-bit security.

In [ ]:
class GenderPayGapSurveyParticipant:
    def submit_salary(self, salary, gender, server):
        """Submits an encrypted survey response to the server"""

        # YOUR CODE HERE
        raise NotImplementedError()

class GenderPayGapSurveyServer:
    def __init__(self):
        self.salaries = []
        self.genders = []
        self.sk, self.pk = keygen(32)
    
    def get_public_key(self):
        return self.pk
        
    def submit_salary(self, ct_salary_vector, ct_gender_vector):
        """Store an entry in the survey"""

        # YOUR CODE HERE
        raise NotImplementedError()
    
    def show_salaries(self):
        """Display the (encrypted) submitted salaries"""
        return self.salaries
    
    def compute_average_salaries(self):
        """Tally the results, decrypt, and return a 2-tuple: (average female salary, average male salary)"""
        
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE
s = GenderPayGapSurveyServer()
GenderPayGapSurveyParticipant().submit_salary(10000, 'Male', s)
GenderPayGapSurveyParticipant().submit_salary(30000, 'Female', s)
GenderPayGapSurveyParticipant().submit_salary(15000, 'Male', s)
GenderPayGapSurveyParticipant().submit_salary(20000, 'Female', s)
assert s.compute_average_salaries() == (25000.0, 12500.0)

## Question 5 (10 points)

In 2-5 sentences, answer the following:

- What is one way a malicious *survey server* could break the rules of the survey?
- What are two ways a malicious *survey participant* could break the rules of the survey?

YOUR ANSWER HERE